In [1]:
import pandas as pd  
from sklearn.linear_model import LinearRegression 

# 데이터 불러오기
df = pd.read_csv('water_survive.csv') 

# 'ph'와 'Sulfate' 간의 상관관계 계산
correlation = df['Solids'].corr(df['Sulfate'])
print(f'ph와 Sulfate 간의 상관관계: {correlation}')

# 'Sulfate'의 결측값 개수 확인
missing_values = df['Sulfate'].isnull().sum()
print(f'Sulfate 열의 결측값 개수: {missing_values}')


# 'Sulfate'의 결측값을 'ph'를 기준으로 회귀 모델을 사용하여 채움
# 각 'Origin' 그룹에 대해 선형 회귀 모델을 학습하고 결측값을 예측하여 채움
for Origin, group in df.groupby('Origin'):
    train = group.dropna(subset=['Sulfate', 'Solids'])
    test = group[group['Sulfate'].isnull() & group['Solids'].notnull()]
    if not train.empty and not test.empty:
        X_train = train[['Solids']]
        y_train = train['Sulfate']
        X_test = test[['Solids']]
        
        model = LinearRegression()
        model.fit(X_train, y_train)
        
        predicted = model.predict(X_test)
        
        df.loc[test.index, 'Sulfate'] = predicted

# 다시 결측값 확인
missing_values_after = df['Sulfate'].isnull().sum()
print(f'Sulfate 열의 결측값 개수 (채운 후): {missing_values_after}')

# 이상치 제거
Q1 = df[['Solids', 'Sulfate']].quantile(0.25)
Q3 = df[['Solids', 'Sulfate']].quantile(0.75)
IQR = Q3 - Q1
# 이상치 조건
condition_ph = (df['Solids'] >= Q1['Solids'] - 1.5 * IQR['Solids']) & (df['Solids'] <= Q3['Solids'] + 1.5 * IQR['Solids'])
condition_sulfate = (df['Sulfate'] >= Q1['Sulfate'] - 1.5 * IQR['Sulfate']) & (df['Sulfate'] <= Q3['Sulfate'] + 1.5 * IQR['Sulfate'])

condition = condition_ph & condition_sulfate

df_cleaned = df[condition]

df_cleaned_final=df_cleaned.dropna()
# CSV 파일로 저장
df_cleaned_final.to_csv('Solids_Sulfate_corr.csv', index=False)
print(f'최종 데이터프레임 형태: {df_cleaned.shape}')



ph와 Sulfate 간의 상관관계: -0.1718042747306018
Sulfate 열의 결측값 개수: 781
Sulfate 열의 결측값 개수 (채운 후): 35
최종 데이터프레임 형태: (2963, 11)


In [ ]:
import pandas as pd  
import numpy as np  
from scipy.stats import kurtosis

# 데이터 불러오기
df = pd.read_csv('ph_Sulfate_corr.csv')
print(df.head())

# 왜도 계산, 출력
skewness = df.select_dtypes(include=[np.number]).skew()
print("왜도:")
print(skewness)

# 첨도 계산, 출력
kurtosis_values = df.select_dtypes(include=[np.number]).apply(kurtosis, fisher=True)
print("첨도:")
print(kurtosis_values)


  Origin       Solids  Chloramines     Sulfate  Conductivity  Organic_carbon  \
0  river  18630.05786     6.635246  333.550915    592.885359       15.180013   
1   lake  19909.54173     9.275884  332.310799    418.606213       16.868637   
2   lake  17978.98634     6.546600  310.135738    398.410813       11.558279   
3   lake  28748.68774     7.544869  326.678363    280.467916        8.399735   
4   lake  28749.71654     7.513408  393.663395    283.651634       13.789695   

   Trihalomethanes  Turbidity    Hardness         ph  Survivability  
0         4.048808   4.500656  129.422921   3.716080              0  
1         4.210943   3.055934  224.236259   8.099124              0  
2         3.496447   4.075075  181.101509   9.092223              0  
3         4.023884   2.559708  188.313324   5.584087              0  
4         8.899454   2.672989  248.071735  10.223862              0  
왜도:
Solids             0.583242
Chloramines        0.032718
Sulfate            0.085345
Conductivit